In [ ]:
import cv2
import copy
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Conv2D,BatchNormalization,GaussianNoise
from keras.layers import MaxPool2D,ZeroPadding2D,MaxPooling2D
from keras.layers import Flatten,Activation
from keras.layers import Dense,Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam,SGD
from keras.callbacks import EarlyStopping
from keras import initializers
import numpy as np
from tensorflow.keras import regularizers

from glob import glob
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.metrics import auc,roc_curve
import cv2

In [ ]:
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau

In [ ]:
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

In [ ]:
#img=cv2.imread("../input/chest_xray/chest_xray/train/PNEUMONIA/person1438_bacteria_3721.jpeg")
#img.shape

In [ ]:
target=32*8
batchsize=128

In [ ]:
classifier=Sequential()

classifier.add(Conv2D(16, (3, 3),kernel_regularizer=regularizers.l2(0.00001), input_shape = (target, target, 3), activation = 'relu'))
classifier.add(BatchNormalization(axis=-1))
#classifier.add(MaxPooling2D(pool_size = (4, 4)))

classifier.add(Conv2D(32, (3, 3),kernel_regularizer=regularizers.l2(0.00001), activation = 'relu'))
classifier.add(BatchNormalization(axis=-1))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

classifier.add(Conv2D(32, (5, 5), kernel_regularizer=regularizers.l2(0.00001),activation = 'relu'))
classifier.add(BatchNormalization(axis=-1))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

classifier.add(Dropout(rate=0.5))

classifier.add(Flatten())

classifier.add(Dense(units = 256,kernel_regularizer=regularizers.l2(0.00001), activation = 'relu'))
classifier.add(BatchNormalization(axis=-1))

classifier.add(Dropout(rate=0.80))

classifier.add(Dense(units = 1, activation = 'sigmoid'))

In [ ]:
#classifier.summary()

In [ ]:
classifier.compile(optimizer = Adam(0.0001) , loss = "binary_crossentropy", metrics=["acc"])

In [ ]:
train_datagen = ImageDataGenerator(rescale=1. / 255)

test_datagen = ImageDataGenerator(rescale=1. / 255)

training_set = train_datagen.flow_from_directory('../input/chest_xray/chest_xray/train',
  target_size = (target, target),
  batch_size = batchsize,
  class_mode='binary')

validation_set = test_datagen.flow_from_directory('../input/chest_xray/chest_xray/val',
  target_size = (target, target),
  batch_size = batchsize,
  class_mode='binary')

test_set = test_datagen.flow_from_directory('../input/chest_xray/chest_xray/test',
  target_size = (target, target),
  batch_size = batchsize,
  class_mode='binary')

In [ ]:
output=classifier.fit_generator(training_set,validation_data=validation_set,validation_steps=16,
                           steps_per_epoch =5216//batchsize,
                           epochs = 10)

In [ ]:
plt.plot(output.history['acc'])
plt.plot(output.history['val_acc'])
plt.title('classifier accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(output.history['loss'])
plt.plot(output.history['val_loss'])
plt.title('classifier loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
score=classifier.evaluate_generator(test_set,steps=1)

print('custom loss: ' + str(score[0]))
print('custom accu: ' + str(score[1]))

In [ ]:
from tensorflow.keras.preprocessing import image

from glob import glob

In [ ]:
test_pne=r"../input/chest_xray/chest_xray/test/PNEUMONIA/*.jpeg"
test_normal=r"../input/chest_xray/chest_xray/test/NORMAL/*.jpeg"

pngs_pne=glob(test_pne)

pngs_normal=glob(test_normal)

len(pngs_pne)

In [ ]:
pngs_pne[2]

In [ ]:
y_hat=list()

y_true=list()

for i in range(len(pngs_pne)):
    
    test_image = image.load_img(pngs_pne[i] ,target_size=(target,target))
    test_image = np.true_divide(test_image,[255.0],out=None)
# Changing the input of the size...
    test_image = image.img_to_array(test_image)

# Adding a new dimension (the placement of the image in the batchsize)
    test_image = np.expand_dims(test_image, axis=0)

    predic_classes = classifier.predict_classes(test_image)
    y_hat.append(predic_classes[0])
    
    y_true.append(1)
    
for i in range(len(pngs_normal)):
    
    test_image = image.load_img(pngs_normal[i],target_size= (target,target))

    test_image = np.true_divide(test_image,[255.0],out=None)
# Changing the input of the size...
    test_image = image.img_to_array(test_image)

# Adding a new dimension (the placement of the image in the batchsize)
    test_image = np.expand_dims(test_image, axis=0)

    predic_classes = classifier.predict_classes(test_image)
    y_hat.append(predic_classes[0])
    
    y_true.append(0)

In [ ]:
from sklearn.metrics import auc,roc_curve

classi_report=classification_report(y_true, y_hat)

fpr_keras, tpr_keras, thresholds_keras = roc_curve(y_true, y_hat)

auc_curve=auc(fpr_keras, tpr_keras)

In [ ]:
cm=confusion_matrix(y_hat,y_true)

test_accuracy=(cm[0,0]+cm[1,1])/(cm[0,0]+cm[1,1]+cm[1,0]+cm[0,1])

In [ ]:
print('auc score is: '+str(auc_curve))

print('predict accuracy: '+str(test_accuracy))

print(cm)

print(classi_report)

In [ ]:
training_set.class_indices